In [1]:
import os, torch, copy, cv2, sys, random
# from datetime import datetime, timezone, timedelta
import wandb

from PIL import Image
import numpy as np
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms

import torchvision.models as models


import albumentations as A
from albumentations.pytorch import ToTensorV2

from sklearn.model_selection import KFold

In [2]:
RANDOM_SEED = 2022

torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

In [3]:
DATA_DIR = '../data'
NUM_CLS = 2

os.environ['CUDA_VISIBLE_DEVICES'] = "0"

DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'현재 DEVICE : {DEVICE}')

현재 DEVICE : cuda


In [4]:
class CustomDataset(Dataset):
    def __init__(self, data_dir, input_shape, transform):
        self.data_dir = data_dir
        self.transform = transform
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        # Dataset split
        # if self.mode == 'train':
        #     self.db = self.db[:int(len(self.db) * 0.9)]
        # elif self.mode == 'val':
        #     self.db = self.db[int(len(self.db) * 0.9):]
        #     self.db.reset_index(inplace=True)
        # else:
        #     print(f'!!! Invalid split {self.mode}... !!!')
            
        # Transform function
        self.transform = transform

    def data_loader(self):
        print('Loading dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        # (COVID : 1, No : 0)
        db = pd.read_csv(os.path.join(self.data_dir, 'train.csv'))
        
        return db

    def __len__(self):
        return len(self.db)

    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])

        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'train',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        cvimg = np.invert(cvimg)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])

        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))
        

        return trans_image, data['COVID']


In [5]:
class LossEarlyStopper():
    """Early stopper
    
    Attributes:
        patience (int): loss가 줄어들지 않아도 학습할 epoch 수
        patience_counter (int): loss 가 줄어들지 않을 때 마다 1씩 증가, 감소 시 0으로 리셋
        min_loss (float): 최소 loss
        stop (bool): True 일 때 학습 중단
    """

    def __init__(self, patience: int)-> None:
        self.patience = patience

        self.patience_counter = 0
        self.min_loss = np.Inf
        self.stop = False
        self.save_model = False

    def check_early_stopping(self, loss: float)-> None:
        """Early stopping 여부 판단"""  

        if self.min_loss == np.Inf:
            self.min_loss = loss
            return None

        elif loss > self.min_loss:
            self.patience_counter += 1
            msg = f"Early stopping counter {self.patience_counter}/{self.patience}"

            if self.patience_counter == self.patience:
                self.stop = True
                
        elif loss <= self.min_loss:
            self.patience_counter = 0
            self.save_model = True
            msg = f"Validation loss decreased {self.min_loss} -> {loss}"
            self.min_loss = loss
        
        print(msg)
    

class Trainer():
    def __init__(self, loss_fn, model, device, metric_fn, optimizer=None, scheduler=None):
        self.loss_fn = loss_fn
        self.model = model
        self.device = device
        self.metric_fn = metric_fn
        self.optimizer = optimizer
        self.scheduler = scheduler
        
    def train_epoch(self, dataloader, epoch_index):

        # train mode
        self.model.train()
        
        train_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []

        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()
            
            # predict
            pred = self.model(img)
            
            # loss
            loss = loss_fn(pred[:, 1], label)

            self.optimizer.zero_grad()
            loss.backward()
            self.optimizer.step()
            self.scheduler.step()
            
            train_total_loss += loss.item()
            
            prob_lst.extend(pred[:, 1].cpu().tolist())
            target_lst.extend(label.cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
            
        self.train_mean_loss = train_total_loss / batch_index
        self.train_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        
        # epoch msg
        print(f'Epoch {epoch_index}, Train loss: {self.train_mean_loss:.4f}, Acc: {self.train_score:.4f}, F1-Macro: {f1:.4f}')
        
    def validate_epoch(self, dataloader, epoch_index):
        self.model.eval()
        val_total_loss = 0
        target_lst = []
        pred_lst = []
        prob_lst = []
    
        
        for batch_index, (img, label) in enumerate(dataloader):
            img = img.to(self.device)
            label = label.to(self.device).float()

            pred = self.model(img)
            
            loss = self.loss_fn(pred[:, 1], label)
            val_total_loss += loss.item()
            
            prob_lst.extend(pred[:, 1].cpu().tolist())
            pred_lst.extend(pred.argmax(dim=1).cpu().tolist())
            target_lst.extend(label.cpu().tolist())
        
        self.val_mean_loss = val_total_loss / batch_index
        self.validation_score, f1 = self.metric_fn(y_pred=pred_lst, y_answer=target_lst)
        print(f'Epoch {epoch_index}, Val loss: {self.val_mean_loss:.4f}, Acc: {self.validation_score:.4f}, F1-Macro: {f1:.4f}')        

In [6]:
from sklearn.metrics import accuracy_score, f1_score

def get_metric_fn(y_pred, y_answer):
    """ 성능을 반환하는 함수"""
    
    assert len(y_pred) == len(y_answer), 'The size of prediction and answer are not same.'
    accuracy = accuracy_score(y_answer, y_pred)
    f1 = f1_score(y_answer, y_pred, average='macro')
    return accuracy, f1

In [7]:
class VGG11_bn(nn.Module):
    def __init__(self, num_classes):
        super(VGG11_bn, self).__init__()
        self.model = models.vgg11_bn(pretrained=False)
        self.model.classifier[6] = nn.Linear(self.model.classifier[6].in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.model(x)
        output = self.softmax(x)
        
        return output

class VGG13_bn(nn.Module):
    def __init__(self, num_classes):
        super(VGG13_bn, self).__init__()
        self.model = models.vgg13_bn(pretrained=False)
        self.model.classifier[6] = nn.Linear(self.model.classifier[6].in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.model(x)
        output = self.softmax(x)
        
        return output
    
class VGG13(nn.Module):
    def __init__(self, num_classes):
        super(VGG13, self).__init__()
        self.model = models.vgg13(pretrained=False)
        self.model.classifier[6] = nn.Linear(self.model.classifier[6].in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.model(x)
        output = self.softmax(x)
        
        return output
    
    
class VGG16_bn(nn.Module):
    def __init__(self, num_classes):
        super(VGG16_bn, self).__init__()
        self.model = models.vgg16_bn(pretrained=False)
        self.model.classifier[6] = nn.Linear(self.model.classifier[6].in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.model(x)
        output = self.softmax(x)
        
        return output

In [8]:
class Resnet18(nn.Module):
    def __init__(self, num_classes):
        super(Resnet18, self).__init__()
        self.model = models.resnet18(pretrained=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.model(x)
        output = self.softmax(x)
        
        return output
    
class Resnet34(nn.Module):
    def __init__(self, num_classes):
        super(Resnet34, self).__init__()
        self.model = models.resnet34(pretrained=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.model(x)
        output = self.softmax(x)
        
        return output

class Resnet50(nn.Module):
    def __init__(self, num_classes):
        super(Resnet50, self).__init__()
        self.model = models.resnet50(pretrained=False)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)
        self.softmax = nn.Softmax(dim=1)
        
    def forward(self, x):
        x = self.model(x)
        output = self.softmax(x)
        
        return output

In [19]:
INPUT_SHAPE=256

mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

original = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(INPUT_SHAPE),
    transforms.Normalize(mean, std),  
    transforms.Grayscale(num_output_channels=3)
])

original2 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize(INPUT_SHAPE),
    transforms.Normalize(mean, std),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.Grayscale(num_output_channels=3)
])

original3 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean, std),
    transforms.Grayscale(num_output_channels=3),
    transforms.RandomRotation(degrees=(0, 180)),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.CenterCrop(size=245),
    transforms.Resize(INPUT_SHAPE)
    
])


In [20]:
# Load dataset & dataloader
dataset = CustomDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE, transform=original2)

# dataset = torch.utils.data.ConcatDataset([original_train, original_train2])
# valid_dataset = torch.utils.data.ConcatDataset([original_valid])

print(len(dataset))

Loading dataset..
646


In [22]:
EPOCHS = 30
LEARNING_RATE = 0.0003
EARLY_STOPPING_PATIENCE = 15
BATCH_SIZE = 32

name = 'Resnet18_invert_kfold'

config = {
        'epochs': EPOCHS, 
        'learning rate': LEARNING_RATE, 
        'batch_size': BATCH_SIZE,
        'INPUT_SHAPE':INPUT_SHAPE,
        'EARLY_STOPPING_PATIENCE':EARLY_STOPPING_PATIENCE,
        'augmentation': ['normalize', 'distortion', 'crop', 'rotate', 'flip'],

}




# Load Model
model = Resnet18(2).to(DEVICE)

# Set optimizer, scheduler, loss function, metric function
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)

loss_fn = nn.BCELoss()
metric_fn = get_metric_fn


# Set trainer


# Set earlystopper
early_stopper = LossEarlyStopper(patience=EARLY_STOPPING_PATIENCE)


CV = 5
kfold = KFold(n_splits=CV)

for fold, (train_idx, valid_idx) in enumerate(kfold.split(dataset)):
    print(f'--------------fold No.{fold}--------------')
    train_subsampler = torch.utils.data.SubsetRandomSampler(train_idx)
    valid_subsampler = torch.utils.data.SubsetRandomSampler(valid_idx)
    
    
    trainloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=train_subsampler)
    validloader = torch.utils.data.DataLoader(dataset, batch_size=BATCH_SIZE, sampler=valid_subsampler)
    
    model = Resnet18(2).to(DEVICE)
    
    scheduler =  optim.lr_scheduler.OneCycleLR(optimizer=optimizer, pct_start=0.1, div_factor=1e5, max_lr=0.0001, epochs=EPOCHS, steps_per_epoch=len(trainloader))
    trainer = Trainer(loss_fn, model, DEVICE, metric_fn, optimizer, scheduler)
    
    print('Train set samples:',len(trainloader),  'Val set samples:', len(validloader))
    wandb.init(
        config=config,
        project='CT task',
        tags=['resnet'],
        group='Kfold',
        name= str(fold)+'_'+name,
        notes='normalize',
        save_code=True,
    )
    for epoch_index in tqdm(range(EPOCHS)):

        trainer.train_epoch(trainloader, epoch_index)
        trainer.validate_epoch(validloader, epoch_index)
        wandb.log({
            'train loss' : trainer.train_mean_loss,
            'train acc score' : trainer.train_score,
            'valid loss' : trainer.val_mean_loss,
            'valid acc score' : trainer.validation_score
        })
        # early_stopping check
        early_stopper.check_early_stopping(loss=trainer.val_mean_loss)
        print(early_stopper.save_model)
        if early_stopper.stop:
            print('Early stopped')
            break

        if early_stopper.save_model:
            check_point = {
                'model': model.state_dict(),
                'optimizer': optimizer.state_dict(),
                'scheduler': scheduler.state_dict()
            }
            torch.save(check_point, f'best/{fold}_{name}best.pt')
            early_stopper.save_model = False
        
    wandb.finish(quiet=True)


--------------fold No.0--------------
Train set samples: 17 Val set samples: 5


train acc score,▁▂▂▆▄▂▅█▄
train loss,▆█▆█▆▆▄▄▁
valid acc score,▁▂▆▃▅█▂▅▇
valid loss,▆▅▄█▇▁▄▅▅
train acc score,0.53295
train loss,0.72874
valid acc score,0.56923
valid loss,0.85643


  0% 0/30 [00:00<?, ?it/s]

Epoch 0, Train loss: 0.7417, Acc: 0.5465, F1-Macro: 0.3534


  3% 1/30 [01:38<47:26, 98.16s/it]

...

Epoch 6, Val loss: 0.9456, Acc: 0.4615, F1-Macro: 0.3158
Early stopping counter 6/15
False
Epoch 7, Train loss: 0.7504, Acc: 0.5465, F1-Macro: 0.3534


 27% 8/30 [12:58<35:46, 97.58s/it]

Epoch 7, Val loss: 0.9220, Acc: 0.4615, F1-Macro: 0.3158
Early stopping counter 7/15
False


 27% 8/30 [13:11<36:16, 98.95s/it]


KeyboardInterrupt: 

In [ ]:
TRAINED_MODEL_PATH = 'best/'+name+'best.pt'

In [ ]:
class TestDataset(Dataset):
    def __init__(self, data_dir, input_shape):
        self.data_dir = data_dir
        self.input_shape = input_shape
        
        # Loading dataset
        self.db = self.data_loader()
        
        
        mean = (0.485, 0.456, 0.406)
        std = (0.229, 0.224, 0.225)
        # Transform function
        self.transform = transforms.Compose([transforms.Resize(self.input_shape),
                                             transforms.ToTensor(),
                                             transforms.Normalize(mean, std)])

    def data_loader(self):
        print('Loading test dataset..')
        if not os.path.isdir(self.data_dir):
            print(f'!!! Cannot find {self.data_dir}... !!!')
            sys.exit()
        
        db = pd.read_csv(os.path.join(self.data_dir, 'sample_submission.csv'))
        return db
    
    def __len__(self):
        return len(self.db)
    
    def __getitem__(self, index):
        data = copy.deepcopy(self.db.loc[index])
        
        # Loading image
        cvimg = cv2.imread(os.path.join(self.data_dir,'test',data['file_name']), cv2.IMREAD_COLOR | cv2.IMREAD_IGNORE_ORIENTATION)
        cvimg = np.invert(cvimg)
        if not isinstance(cvimg, np.ndarray):
            raise IOError("Fail to read %s" % data['file_name'])
        
        # Preprocessing images
        trans_image = self.transform(Image.fromarray(cvimg))

        return trans_image, data['file_name']

In [ ]:
# Load dataset & dataloader
test_dataset = TestDataset(data_dir=DATA_DIR, input_shape=INPUT_SHAPE)
test_dataloader = DataLoader(dataset=test_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [ ]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

In [ ]:
model.load_state_dict(torch.load(TRAINED_MODEL_PATH)['model'])

# Prediction
file_lst = []
pred_lst = []
prob_lst = []
model.eval()
with torch.no_grad():
    for batch_index, (img, file_num) in tqdm(enumerate(test_dataloader)):
        img = img.to(DEVICE)
        pred = model(img)
        
        file_lst.extend(list(file_num))
        pred_lst.extend(pred.argmax(dim=1).tolist())
        prob_lst.extend(pred[:, 1].tolist())

In [ ]:
df = pd.DataFrame({'file_name':file_lst, 'COVID':pred_lst})
# df.sort_values(by=['file_name'], inplace=True)
df.to_csv('result/prediction_' + name+'.csv', index=False)